In [77]:
import pandas as pd
import numpy as np
import pickle

In [3]:
ls DATA

Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv


# Data Import

In [32]:
df = pd.read_csv('DATA/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv')
df.head(3)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,245762.0,...,1292776.0,1288753.0,1269532.0,1243884.0,1211977.0,1197322.0,1185428.0,1179938.0,1175379.0,1173231.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,209547.0,...,487111.0,486300.0,486154.0,487283.0,488823.0,489789.0,489865.0,490118.0,491195.0,493022.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,230594.0,...,1080810.0,1099111.0,1117633.0,1130101.0,1129983.0,1138594.0,1143043.0,1147409.0,1149477.0,1155724.0


The dataframe is in the wide format. I want it to have a column per region and row per timestamp. I will also filter the location down to include only New York. First let's check if our region's are unique.

In [33]:
df.RegionName.value_counts().sort_values(ascending = False)[0:5]

65536    1
2081     1
33040    1
79911    1
77830    1
Name: RegionName, dtype: int64

## Export Metadata
Let's separate out the region meta info so we can reference it later.

In [34]:
meta = df.iloc[:, 0:9]

In [35]:
#import os
#os.mkdir('PKL')

In [36]:
# saving meta data
with open('PKL/meta.pkl', 'wb') as fp:
    pickle.dump(meta, fp)

## Subset
Subsetting to only Queens for now.

In [41]:
queens = df[df.CountyName == 'Queens County']

In [42]:
meta_cols = list(df.columns[1:9])
queens = queens.drop(meta_cols, axis = 1)

In [43]:
queens.shape

(54, 297)

In [46]:
queens.columns[queens.isnull().sum() != 0][-1]

'2003-08-31'

It seems like we have full data of all queens zipcode starting from 2003 September. Let's cap it at that.

In [48]:
queens = queens.dropna(axis = 1)

## Transpose
Now I'll change the format to the wide format.

In [60]:
queens = queens.set_index('RegionID').transpose()

## Fix Datetime
Let's convert the index to datetime.

In [64]:
queens.index = pd.DatetimeIndex(queens.index)

In [74]:
queens.columns.name = None

## Exporting 
Now let's export the dataframe.

In [76]:
with open('PKL/timeseries_queens.pkl', 'wb') as fp:
    pickle.dump(queens, fp)